In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Experimentos - Seção 5.2: Comparação com Métodos de Aproximação\n",
    "\n",
    "Comparação sistemática entre FFA Formal, LIME, SHAP e Permutation Importance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import pandas as pd\n",
    "\n",
    "# Adiciona src ao path\n",
    "sys.path.append('../src')\n",
    "\n",
    "from experiments.section_5_2 import run_section_5_2\n",
    "from utils.metrics import calculate_correlations, calculate_ranking_metrics\n",
    "from utils.visualization import plot_attribution_comparison\n",
    "\n",
    "# Configurações de visualização\n",
    "plt.style.use('default')\n",
    "sns.set_palette(\"husl\")\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carregamento dos Resultados da Seção 5.1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"📂 CARREGANDO RESULTADOS DA SEÇÃO 5.1\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "import pickle\n",
    "\n",
    "try:\n",
    "    with open('../data/results/section_5_1_results.pkl', 'rb') as f:\n",
    "        results_5_1 = pickle.load(f)\n",
    "    print(\"✅ Resultados da Seção 5.1 carregados com sucesso!\")\n",
    "    \n",
    "    # Mostra resumo\n",
    "    for dataset_name, data in results_5_1.items():\n",
    "        perf = data['performance']\n",
    "        print(f\"\\n📊 {dataset_name.upper()}:\")\n",
    "        print(f\"   • Acurácia Teste: {perf['test_accuracy']:.3f}\")\n",
    "        print(f\"   • Instâncias: {len(data['sample_indices'])}\")\n",
    "        \n",
    "except FileNotFoundError:\n",
    "    print(\"❌ Arquivo de resultados não encontrado!\")\n",
    "    print(\"💡 Execute primeiro o notebook 02_experimentos_5_1.ipynb\")\n",
    "    results_5_1 = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Execução dos Experimentos da Seção 5.2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if results_5_1 is not None:\n",
    "    print(\"\\n🚀 EXECUTANDO EXPERIMENTOS DA SEÇÃO 5.2\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    results_5_2 = run_section_5_2(results_5_1)\n",
    "    \n",
    "    # Salva resultados\n",
    "    with open('../data/results/section_5_2_results.pkl', 'wb') as f:\n",
    "        pickle.dump(results_5_2, f)\n",
    "    print(\"\\n✅ Resultados da Seção 5.2 salvos em:'../data/results/section_5_2_results.pkl'\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Análise Comparativa Detalhada"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if results_5_1 is not None:\n",
    "    print(\"📊 ANÁLISE COMPARATIVA DETALHADA\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    all_correlations = []\n",
    "    \n",
    "    for dataset_name, data in results_5_2.items():\n",
    "        print(f\"\\n🎯 DATASET: {dataset_name.upper()}\")\n",
    "        print(\"-\" * 40)\n",
    "        \n",
    "        metrics_data = data['metrics']\n",
    "        \n",
    "        # Calcula estatísticas agregadas\n",
    "        correlations_ffa_lime = []\n",
    "        correlations_ffa_shap = []\n",
    "        correlations_lime_shap = []\n",
    "        top_feature_agreements = []\n",
    "        \n",
    "        for instance_idx, metrics in metrics_data.items():\n",
    "            corrs = metrics['correlations']\n",
    "            ranking = metrics['ranking_metrics']\n",
    "            \n",
    "            correlations_ffa_lime.append(corrs['ffa_vs_lime'])\n",
    "            correlations_ffa_shap.append(corrs['ffa_vs_shap'])\n",
    "            correlations_lime_shap.append(corrs['lime_vs_shap'])\n",
    "            top_feature_agreements.append(ranking['top_feature_agreement'])\n",
    "            \n",
    "            # Armazena para análise geral\n",
    "            all_correlations.append({\n",
    "                'dataset': dataset_name,\n",
    "                'instance': instance_idx,\n",
    "                'ffa_vs_lime': corrs['ffa_vs_lime'],\n",
    "                'ffa_vs_shap': corrs['ffa_vs_shap'],\n",
    "                'lime_vs_shap': corrs['lime_vs_shap']\n",
    "            })\n",
    "        \n",
    "        # Estatísticas do dataset\n",
    "        print(f\"📈 Correlações Médias (Kendall's Tau):\")\n",
    "        print(f\"   • FFA vs LIME:  {np.mean(correlations_ffa_lime):.3f} (±{np.std(correlations_ffa_lime):.3f})\")\n",
    "        print(f\"   • FFA vs SHAP:  {np.mean(correlations_ffa_shap):.3f} (±{np.std(correlations_ffa_shap):.3f})\")\n",
    "        print(f\"   • LIME vs SHAP: {np.mean(correlations_lime_shap):.3f} (±{np.std(correlations_lime_shap):.3f})\")\n",
    "        \n",
    "        print(f\"\\n🎯 Concordância no Top Feature:\")\n",
    "        agreement_rate = np.mean(top_feature_agreements)\n",
    "        print(f\"   • Taxa de concordância: {agreement_rate:.1%} ({sum(top_feature_agreements)}/{len(top_feature_agreements)} instâncias)\")\n",
    "        \n",
    "        # Análise de consistência\n",
    "        high_agreement_ffa_lime = sum(1 for corr in correlations_ffa_lime if corr > 0.5)\n",
    "        high_agreement_ffa_shap = sum(1 for corr in correlations_ffa_shap if corr > 0.5)\n",
    "        \n",
    "        print(f\"\\n🔍 Análise de Consistência:\")\n",
    "        print(f\"   • Alta concordância FFA-LIME:  {high_agreement_ffa_lime}/{len(correlations_ffa_lime)} instâncias\")\n",
    "        print(f\"   • Alta concordância FFA-SHAP:  {high_agreement_ffa_shap}/{len(correlations_ffa_shap)} instâncias\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualização das Comparações"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if results_5_1 is not None:\n",
    "    print(\"📈 VISUALIZAÇÃO DAS COMPARAÇÕES\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    # Cria diretório para gráficos\n",
    "    os.makedirs('../data/results/plots', exist_ok=True)\n",
    "    \n",
    "    for dataset_name, data in results_5_2.items():\n",
    "        sample_indices = data['sample_indices']\n",
    "        \n",
    "        for i, instance_idx in enumerate(sample_indices):\n",
    "            print(f\"\\n📊 Gerando gráfico: {dataset_name} - Instância {instance_idx}\")\n",
    "            \n",
    "            # Prepara dados para plotagem\n",
    "            plot_data = {\n",
    "                'feature_names': data['feature_names'],\n",
    "                'formal_attributions': data['formal_attributions'],\n",
    "                'lime_attributions': data['lime_attributions'],\n",
    "                'shap_attributions': data['shap_attributions'],\n",
    "                'permutation_importance': data['permutation_importance']\n",
    "            }\n",
    "            \n",
    "            save_path = f'../data/results/plots/{dataset_name}_instance_{instance_idx}.png'\n",
    "            plot_attribution_comparison(plot_data, dataset_name, i, save_path)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Análise Estatística Agregada"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if results_5_1 is not None:\n",
    "    print(\"📊 ANÁLISE ESTATÍSTICA AGREGADA\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    # Converte para DataFrame\n",
    "    df_correlations = pd.DataFrame(all_correlations)\n",
    "    \n",
    "    # Estatísticas gerais\n",
    "    print(\"\\n📈 ESTATÍSTICAS GERAIS DAS CORRELAÇÕES:\")\n",
    "    print(\"-\" * 40)\n",
    "    \n",
    "    for comparison in ['ffa_vs_lime', 'ffa_vs_shap', 'lime_vs_shap']:\n",
    "        values = df_correlations[comparison]\n",
    "        print(f\"\\n🔍 {comparison.upper()}:\")\n",
    "        print(f\"   • Média: {values.mean():.3f}\")\n",
    "        print(f\"   • Desvio Padrão: {values.std():.3f}\")\n",
    "        print(f\"   • Mínimo: {values.min():.3f}\")\n",
    "        print(f\"   • Máximo: {values.max():.3f}\")\n",
    "        print(f\"   • > 0.5: {len(values[values > 0.5])}/{len(values)} ({len(values[values > 0.5])/len(values):.1%})\")\n",
    "        print(f\"   • < 0.0: {len(values[values < 0.0])}/{len(values)} ({len(values[values < 0.0])/len(values):.1%})\")\n",
    "    \n",
    "    # Análise por dataset\n",
    "    print(\"\\n📊 ANÁLISE POR DATASET:\")\n",
    "    print(\"-\" * 40)\n",
    "    \n",
    "    dataset_stats = df_correlations.groupby('dataset').agg({\n",
    "        'ffa_vs_lime': ['mean', 'std'],\n",
    "        'ffa_vs_shap': ['mean', 'std'],\n",
    "        'lime_vs_shap': ['mean', 'std']\n",
    "    }).round(3)\n",
    "    \n",
    "    print(dataset_stats)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Visualização das Correlações"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if results_5_1 is not None:\n",
    "    print(\"📈 VISUALIZAÇÃO DAS CORRELAÇÕES\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    # Boxplot das correlações\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 6))\n",
    "    \n",
    "    comparisons = ['ffa_vs_lime', 'ffa_vs_shap', 'lime_vs_shap']\n",
    "    titles = ['FFA vs LIME', 'FFA vs SHAP', 'LIME vs SHAP']\n",
    "    \n",
    "    for idx, (comp, title) in enumerate(zip(comparisons, titles)):\n",
    "        # Prepara dados\n",
    "        data_to_plot = []\n",
    "        labels = []\n",
    "        \n",
    "        for dataset_name in df_correlations['dataset'].unique():\n",
    "            dataset_data = df_correlations[df_correlations['dataset'] == dataset_name][comp]\n",
    "            data_to_plot.append(dataset_data)\n",
    "            labels.append(dataset_name)\n",
    "        \n",
    "        # Boxplot\n",
    "        bp = axes[idx].boxplot(data_to_plot, labels=labels, patch_artist=True)\n",
    "        \n",
    "        # Customiza cores\n",
    "        colors = ['lightblue', 'lightcoral']\n",
    "        for patch, color in zip(bp['boxes'], colors):\n",
    "            patch.set_facecolor(color)\n",
    "        \n",
    "        axes[idx].set_title(f'{title}\\nCorrelação de Kendall', fontweight='bold')\n",
    "        axes[idx].set_ylabel('Correlação')\n",
    "        axes[idx].grid(axis='y', alpha=0.3)\n",
    "        axes[idx].set_ylim(-1, 1)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Heatmap de correlações médias\n",
    "    print(\"\\n🔥 HEATMAP DE CORRELAÇÕES MÉDIAS\")\n",
    "    \n",
    "    # Calcula matriz de correlações médias\n",
    "    correlation_matrix = df_correlations.groupby('dataset')[comparisons].mean()\n",
    "    \n",
    "    fig, ax = plt.subplots(figsize=(10, 6))\n",
    "    sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, \n",
    "                vmin=-1, vmax=1, ax=ax, fmt='.3f')\n",
    "    ax.set_title('Correlações Médias por Dataset e Método', fontweight='bold', pad=20)\n",
    "    plt.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Análise de Discordâncias"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if results_5_1 is not None:\n",
    "    print(\"🔍 ANÁLISE DE DISCORDÂNCIAS\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    # Identifica casos de maior discordância\n",
    "    high_disagreement = df_correlations[\n",
    "        (df_correlations['ffa_vs_lime'] < 0) | \n",
    "        (df_correlations['ffa_vs_shap'] < 0)\n",
    "    ]\n",
    "    \n",
    "    print(f\"\\n⚠️  CASOS DE DISCORDÂNCIA (correlação negativa):\")\n",
    "    print(f\"   • Total de instâncias com discordância: {len(high_disagreement)}/{len(df_correlations)}\")\n",
    "    print(f\"   • Taxa de discordância: {len(high_disagreement)/len(df_correlations):.1%}\")\n",
    "    \n",
    "    if len(high_disagreement) > 0:\n",
    "        print(f\"\\n📋 INSTÂNCIAS COM MAIOR DISCORDÂNCIA:\")\n",
    "        for _, row in high_disagreement.iterrows():\n",
    "            print(f\"   • {row['dataset']} - Instância {row['instance']}:\")\n",
    "            print(f\"     FFA vs LIME: {row['ffa_vs_lime']:.3f}, FFA vs SHAP: {row['ffa_vs_shap']:.3f}\")\n",
    "    \n",
    "    # Análise de concordância entre LIME e SHAP\n",
    "    lime_shap_high_agreement = len(df_correlations[df_correlations['lime_vs_shap'] > 0.7])\n",
    "    lime_shap_low_agreement = len(df_correlations[df_correlations['lime_vs_shap'] < 0.3])\n",
    "    \n",
    "    print(f\"\\n🔗 CONCORDÂNCIA LIME vs SHAP:\")\n",
    "    print(f\"   • Alta concordância (>0.7): {lime_shap_high_agreement}/{len(df_correlations)} ({lime_shap_high_agreement/len(df_correlations):.1%})\")\n",
    "    print(f\"   • Baixa concordância (<0.3): {lime_shap_low_agreement}/{len(df_correlations)} ({lime_shap_low_agreement/len(df_correlations):.1%})\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Resumo Executivo e Conclusões"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if results_5_1 is not None:\n",
    "    print(\"📋 RESUMO EXECUTIVO - SEÇÃO 5.2\")\n",